# Resetting iLO manager

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>reset the ilo manager!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li><a href = "https://www.json.org/json-en.html">json </a></li>
    <li> <a href = "https://pypi.org/project/redfish/">redfish</a> </li>
    
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

#from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success">
    <h1> 2. </h1>Below is the overview of reset_ilo function. In this function we are resetting values for an iLO manager.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.

1. In the <b></b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using<b> get_resource_directory module.</b>

    <br>

2. If the resource directory is enabled or we want to force it's non-use then navigate to the directory where managers members data is present. We make two http get requests to obtain the directory where we actually need to perform the reset operation.
    <b>manager_member_response</b> saves the result of final http get request made in this case.
    <br>

3. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if <b>#"Manager"</b> attribute is present inside the record <b>instances[@odata.type]</b>. we save the uri inside of manager_member_response. 
<br>
4. <ul>
    <li>Lastly, we navigate in the <b>manager_member_response</b> by following this path <i>'Actions'->'#Manager.Reset'->'target'</i>to obtained the url at which reset opreation has to be made and save the same inside reset_ilo_uri.</li><li> On the next line we create a dictionary having the value of <i>'Action' as 'Manager.Reset'</i> and make a POST request to server and pass the same dictionary so as to make changes at the <b>reset_ilo_url</b> location.</li>
    </ul>
    
    If the response of this POST request was 400 it means there was some error from server-side while doing the same operation. <br>
    Else there can also be an error which we aren't aware of, in that case we print out the response status code alongwith erro     message .<br>
    Ultimately, if everything went properly we print out the response object along with success message.
    

 </div>

In [2]:
def reset_ilo(_redfishobj, DISABLE_RESOURCE_DIR):

    managers_members_response = None

    resource_instances = _redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#Manager.' in instance['@odata.type']:
                managers_members_uri = instance['@odata.id']
                managers_members_response = _redfishobj.get(managers_members_uri)

    if managers_members_response:
        reset_ilo_uri = managers_members_response.obj['Actions']['#Manager.Reset']['target']
        body = {'Action': 'Manager.Reset'}
        resp = _redfishobj.post(reset_ilo_uri, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success">
    <h1> 3. </h1>This is Main function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> object is being created mainly to log into the system with valid credentials.
     
    <br>

3.  We try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if           Server  is not avaialbe we get an error saying "server unreachable"<br>
    <br>
4. Then we call <b>reset_ilo</b>function with our redfish object and the function does the resetting of manager.<br>
    

5. Lastly, REDFISHOBJ.logout() will take care of logging you out of the ilo.

 </div>

In [ ]:
if __name__ == "__main__":
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = input("Enter password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    reset_ilo(REDFISHOBJ, DISABLE_RESOURCE_DIR)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>